In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.5 Flash > 멀티모달 모드 소개

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgetting-started%2Fintro_gemini_2_5_flash.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| 저자 |
| --- |
| [Eric Dong](https://github.com/gericdong) |
| [Holt Skinner](https://github.com/holtskinner) |
| 번역/편집 |
| [T Kim](https://github.com/aimldl) |

### 목표
Gemini 2.5 Flash 모델의 멀티모달 모드를 사용하는 방법을 배웁니다.

### 멀티모달 모드
- 텍스트 프롬프트에서 텍스트 생성
- 스트리밍 텍스트 생성
- 토큰 수 계산

## 시작하기

### Python용 Google Gen AI SDK 설치

In [1]:
%pip install --upgrade --quiet google-genai

Note: you may need to restart the kernel to use updated packages.


### 노트북 환경 인증 (Colab 전용)
- Vertex AI Workbench 사용자는 생략
- Google Colab에서 이 노트북을 실행하는 경우, 아래 셀을 실행해서 환경을 인증하세요.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 라이브러리 가져오기

In [2]:
from IPython.display import HTML, Image, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    SafetySetting,
    ThinkingConfig,
    Tool,
    ToolCodeExecution,
)

### Vertex AI용 Google Cloud 프로젝트 사용
- Google Cloud 프로젝트에서 [Vertex AI API 활성화](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)합니다.
- Google Cloud 프로젝트의 Vertex AI Workbench Instance에서 실행할 경우, 자동으로 `PROJECT_ID`와 `LOCATION`을 불러옵니다.

In [3]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

print(f"PROJECT_ID={PROJECT_ID}")
print(f"LOCATION={LOCATION}")

PROJECT_ID=vertex-workbench-notebook
LOCATION=us-central1


In [4]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## Gemini 2.5 Flash 모델의 사용

- [Gemini 2.5 Flash](https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-5-flash?hl=ko)의 Model ID를 확인하려면 [여기](https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-5-flash?hl=ko)를 클릭하세요. 

In [10]:
#MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}
MODEL_ID = "gemini-2.5-pro"  # @param {type: "string"}

## 멀티모달 프롬프트 보내기
- Gemini 2.5 Flash는 멀티모달 프롬프트를 지원하는 모델입니다.
- 다양한 소스에서 다음 데이터 유형 중 하나를 포함할 수 있습니다.

### 지원되는 데이터 유형

| 데이터 유형 | 소스 | MIME 유형 |
| :---------- | :------------------------------------ | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 텍스트 | 인라인, 로컬 파일, 일반 URL, Google Cloud Storage | `text/plain` `text/html` |
| 코드 | 인라인, 로컬 파일, 일반 URL, Google Cloud Storage | `text/plain` |
| 문서 | 로컬 파일, 일반 URL, Google Cloud Storage | `application/pdf` |
| 이미지 | 로컬 파일, 일반 URL, Google Cloud Storage | `image/jpeg` `image/png` `image/webp` |
| 비디오 | 로컬 파일, 일반 URL, Google Cloud Storage, YouTube | `video/mp4` `video/mpeg` `video/x-flv` `video/quicktime` `video/mpegps` `video/mpg` `video/webm` `video/wmv` `video/3gpp` |
| 오디오 | 로컬 파일, 일반 URL, Google Cloud Storage | `audio/aac` `audio/flac` `audio/mp3` `audio/m4a` `audio/mpeg` `audio/mpga` `audio/mp4` `audio/opus` `audio/pcm` `audio/wav` `audio/webm` |

멀티모달 사용 사례의 더 많은 예는 [이 노트북](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/intro_multimodal_use_cases.ipynb)을 참조하세요.

### 로컬 이미지 사용하기
`images/mbc_golden_mouse_hall.png`
<img src="images/mbc_golden_mouse_hall.png">

In [11]:
with open("images/mbc_golden_mouse_hall.png", "rb") as f:
    image = f.read()

prompt="이 사진에 대해 설명해죠. 중앙 우측의 TV에 발표 제목이 있는데, 그 제목이 뭐야?"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[     
        Part.from_bytes(data=image, mime_type="image/png"),
        prompt,
    ],
    #config=GenerateContentConfig(
    #    thinking_config=thinking_config,
    #),
)

display(Markdown(response.text))

네, 이 사진에 대해 자세히 설명해 드리겠습니다.

### 사진 설명

이 사진은 비어있는 현대적인 강당, 소극장, 또는 발표회장의 내부를 보여주고 있습니다. 촬영은 무대 쪽에서 객석 전체를 바라보는 구도로 이루어졌습니다.

*   **객석:** 좌석은 계단식으로 배치되어 있으며, 세 구역으로 나뉩니다. 중앙에는 주황색(테라코타색) 좌석이 있고, 양쪽에는 회색 좌석이 있습니다. 모든 좌석은 비어있습니다.
*   **무대와 바닥:** 사진의 맨 앞쪽에는 광택이 나는 짙은 색 바닥과 그 위로 나무 재질의 무대 바닥이 보입니다.
*   **천장과 조명:** 천장은 검은색으로 마감되어 있으며, 수많은 조명 장비, 스포트라이트, 프로젝터 등이 복잡한 구조물에 매달려 있어 전문적인 공연/발표 공간임을 알 수 있습니다. 중앙 상단에서는 매우 밝은 조명이 강한 빛을 발하며 별 모양의 플레어 효과를 만들고 있습니다.
*   **벽:** 양쪽 벽은 음향 조절을 위한 것으로 보이는 구멍이 뚫린 밝은 색의 패널로 덮여 있습니다. 왼쪽 벽에는 아날로그 시계가 걸려 있으며 시간은 약 3시 45분을 가리키고 있습니다.

전체적으로 곧 있을 행사나 발표를 위해 잘 정돈되고 준비된 공간의 느낌을 줍니다.

---

### TV 화면의 발표 제목

사용자께서 질문하신 중앙 우측 상단 스크린에 보이는 발표 제목은 다음과 같습니다.

**"Google Cloud와 생성형 AI 기술 활용법"**

그 아래에는 `Vertex AI, Duet AI, Google Workspace` 등 관련 기술의 이름도 작게 적혀 있습니다.

### YouTube URL에서 비디오 보내기
[YouTube 비디오](https://youtu.be/LxjW4_C3_Cc?si=RIt8xZud9ZulnicC)를 활용해서 비디오 이해
- 주의: 유튜브 링크는 visibility=Public이여야 합니다. 

In [13]:
prompt="""이 비디오의 내용을 시간 순으로 간략히 요약해줘. 
          이 비디오는 어떤 상황인 것 같아? 
          비디오의 주인공은 어디에 몇 시에 도착했어?
          사실 주인공은 이곳에서 있을 세미나 강연의 발표자야.
          첫 번째 세션이 시작하는 시간과 제목, 발표자의 이름을 알려줘.
          발표까지 얼마나 시간이 남았지?"""

video = Part.from_uri(
    file_uri="https://www.youtube.com/watch?v=Se9yQee9ULM",
    mime_type="video/mp4",
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video,
        prompt,
    ],
    #config=GenerateContentConfig(
    #    thinking_config=thinking_config,
    #),
)

display(Markdown(response.text))

다음은 비디오의 내용에 대한 요약과 질문에 대한 답변입니다.

### 비디오 내용 요약

*   **00:00-00:19** 비디오는 MBC 경영센터 건물의 외부에서 시작하여 입구로 향합니다.
*   **00:19-00:46** 주인공이 건물 안으로 들어가자 넓고 현대적인 로비가 나타납니다. 로비에는 높은 천장과 반짝이는 바닥, 큰 기둥들이 있으며 몇몇 사람들이 지나다닙니다.
*   **00:46-01:02** 카메라가 '피지컬 AI와 MBC RTK'라는 제목의 세미나 안내 배너에 초점을 맞춥니다. 배너에는 행사 시간표가 상세히 나와 있습니다.
*   **01:02-01:29** 주인공이 로비를 둘러보며 스타벅스와 투썸플레이스 카페를 발견하고, 조금 일찍 도착해서 커피를 마시러 갈지 고민합니다.
*   **01:29-01:52** 결국 투썸플레이스로 가서 키오스크를 통해 메뉴를 살펴봅니다.

### 상황 추측

이 비디오는 주인공이 MBC 방송센터에서 열리는 '피지컬 AI와 MBC RTK' 세미나에 발표자로 참석하기 위해 도착한 상황으로 보입니다. 세미나 시작 전까지 시간이 남아 로비에 있는 카페에서 시간을 보내려는 것 같습니다.

### 질문에 대한 답변

1.  **주인공은 어디에 몇 시에 도착했나요?**
    *   **장소:** **MBC 방송센터** (상암동)
    *   **도착 시간:** 영상 29초에 보이는 로비 시계가 **오후 1시 40분**경을 가리키고 있으므로, 그 무렵에 도착한 것으로 보입니다.

2.  **첫 번째 세션의 시작 시간, 제목, 발표자는 누구인가요?**
    영상 속 배너에 따르면 첫 번째 세션 정보는 다음과 같습니다.
    *   **시작 시간:** 14:00 (오후 2시)
    *   **제목:** Google Cloud의 생성형 AI Gemini와 GenMedia 최신 기술 동향
    *   **발표자:** Google Cloud 김태영 리드

3.  **발표까지 얼마나 시간이 남았나요?**
    주인공이 오후 1시 40분경에 도착했고, 세미나의 첫 발표는 오후 2시에 시작되므로, 세미나 시작까지는 약 **20분**이 남았습니다. (주인공 본인의 발표가 몇 번째인지는 알 수 없으므로, 세미나 시작 시간을 기준으로 계산했습니다.)

In [14]:
prompt="""이 비디오를 바탕으로 방송용 기획을 하고 싶어. 
          대본 초안 생성: 드라마 장르, 인물 설정만으로 시놉시스와 대본 초안을 자동 생성해줘.
"""

video = Part.from_uri(
    file_uri="https://youtu.be/Se9yQee9ULM",
    mime_type="video/mp4",
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video,
        prompt,
    ],
    #config=GenerateContentConfig(
    #    thinking_config=thinking_config,
    #),
)

display(Markdown(response.text))

네, 알겠습니다. 제공해주신 영상을 바탕으로, 방송용 드라마 기획안을 작성해 보겠습니다. 영상의 배경인 최첨단 방송사(MBC)와 '피지컬 AI'라는 키워드를 중심으로 인물과 이야기를 구성하여 시놉시스와 대본 초안을 생성했습니다.

---

### **드라마 기획안**

#### **1. 기획 의도**

차가운 코드로 이루어진 인공지능이 인간의 물리적 세계와 만나는 지점, '피지컬 AI'.
미디어의 심장부인 방송사에서 벌어지는 이 혁신적인 기술 개발 프로젝트를 통해 우리는 '기술의 진보가 인간을 어디로 이끄는가'라는 근원적인 질문을 던진다.
서로 다른 신념을 가진 천재들의 치열한 경쟁과 협력, 그리고 그 속에서 피어나는 사랑과 성장을 통해 기술과 인간성이 공존하는 미래의 가능성을 탐색하고자 한다.

#### **2. 작품 개요**

*   **제 목**: 코드네임: 제미니 (Code Name: Gemini)
*   **장 르**: 하이테크 오피스 드라마
*   **방송사/플랫폼**: MBC (영상 배경 활용)
*   **로그라인**: 세상을 바꿀 '피지컬 AI' 기술을 두고, 이상을 꿈꾸는 천재 개발자와 현실을 좇는 엘리트 팀장이 방송사의 명운이 걸린 프로젝트 안에서 벌이는 치열한 두뇌 싸움.

---

### **등장인물 설정**

| 이름 | 배역 설정 |
| :--- | :--- |
| **강지훈 (32세)** | **이상을 꿈꾸는 AI 개발자.**<br>사람을 살리는 따뜻한 기술을 꿈꾸는 천재. 겉보기엔 어수룩하지만, 코드 앞에서는 누구보다 날카로워진다. MBC와 기술 기업의 합작 프로젝트 '제미니'의 핵심 개발자로 스카우트된다. 그에게 AI는 단순한 도구가 아닌, 세상과 소통하는 언어다. |
| **최민준 (35세)** | **결과가 전부인 냉철한 엘리트 팀장.**<br>MBC 기술혁신본부 최연소 팀장. '피지컬 AI와 RTK' 기술을 누구보다 먼저 상용화해 최고의 성과를 내는 것이 목표다. 과정보다 결과를 중시하며, 감상적인 지훈의 방식을 비효율적이라 여겨 사사건건 부딪힌다. |
| **이서연 (33세)** | **프로젝트의 균형을 잡는 PD.**<br>'제미니' 프로젝트의 총괄 PD. 기술의 '임팩트'와 대중적 '스토리'를 찾아내는 데 탁월한 능력을 가졌다. 끊임없이 충돌하는 지훈과 민준 사이에서 아슬아슬한 줄타기를 하며 프로젝트를 성공으로 이끌어야 하는 막중한 책임감을 지녔다. |
| **박 이사 (50대)** | **MBC 기술혁신본부 이사.**<br>회사의 미래를 위해 '제미니' 프로젝트를 승인한 야심가. 두 천재 개발자의 가능성을 꿰뚫어 보고 그들의 경쟁을 뒤에서 관망하며 더 큰 그림을 그린다. |

---

### **시놉시스 (Synopsis)**

대한민국 방송의 중심, MBC 상암 사옥. 이곳에서 방송사의 미래를 건 초대형 프로젝트 '코드네임: 제미니'가 극비리에 시작된다. 목표는 정밀 위치 측정 기술(RTK)과 생성형 AI를 결합해, 인간처럼 물리적인 상호작용이 가능한 '피지컬 AI'를 개발하는 것.

프로젝트의 성공을 위해 스카우트된 천재 개발자 강지훈은 AI가 재난 현장에서 인명을 구조하고, 소외된 이들의 눈과 발이 되어주길 꿈꾼다. 하지만 프로젝트를 이끄는 MBC 기술혁신팀장 최민준에게 '제미니'는 오직 압도적인 시장 지배력과 수익을 가져다줄 비즈니스 아이템일 뿐이다.

한편, 이 모든 과정을 콘텐츠로 만들어 대중에게 선보여야 하는 PD 이서연은 기술의 윤리와 상업적 성공이라는 양립하기 힘든 가치 사이에서 고뇌한다. 그녀는 인간적인 이상을 가진 지훈에게 끌리면서도, 프로젝트를 성공시키기 위해선 냉철한 민준의 방식이 필요하다는 것을 안다.

기술 시연회를 앞두고, 지훈과 민준의 갈등은 극에 달한다. 민준은 군사 및 감시용 드론으로의 기술 적용을, 지훈은 재난 대응 및 복지용 로봇으로의 적용을 주장하며 팽팽히 맞선다. 둘의 대립으로 프로젝트는 좌초될 위기에 처하고, 외부 경쟁사의 추격은 거세진다.

결국 서연의 중재와 박 이사의 압박 속에서 두 사람은 마지못해 손을 잡지만, 예상치 못한 외부 해킹으로 '제미니'의 핵심 코드가 유출될 위기에 처하는데…!

과연 이들은 서로의 코드를 이해하고, 세상을 놀라게 할 단 하나의 알고리즘을 완성할 수 있을까?

---

### **대본 초안 (Script Draft)**

**S#1. MBC 상암 경영센터 로비 / 낮**

통유리창으로 눈부신 햇살이 쏟아져 들어오는 거대하고 모던한 로비.
정장을 입은 직원들이 분주하게 오간다. 로비 중앙 기둥에는 영상에서 본 것과 같은 **[피지컬 AI와 MBC RTK - 정밀 위치에 지능을 더하다]** 라는 문구가 적힌 거대한 디지털 배너가 선명하다.

조금은 낡은 백팩을 멘 강지훈(32), 이 거대한 공간이 낯선 듯 두리번거리며 로비로 들어선다. 그의 눈은 경외감과 설렘으로 빛나고 있다. 배너를 한참 동안 넋을 잃고 바라본다.

**지훈**
(혼잣말)
드디어... 진짜 세상으로 나오는구나, 내 코드들이...

그때, 잘 다려진 수트를 입은 최민준(35)이 서류를 들고 빠른 걸음으로 다가온다. 지훈을 위아래로 훑어보는 그의 눈빛엔 노골적인 평가의 시선이 담겨있다.

**민준**
강지훈 씨?

**지훈**
(화들짝 놀라며)
아, 네! 제가 강지훈입니다.

**민준**
최민준입니다. 기술혁신본부 팀장. 시간 맞춰 오셨네요. (손목시계를 툭 치며) 1분 전이긴 하지만.

**지훈**
(멋쩍게 웃으며)
조금 일찍 도착해서... 둘러보고 있었습니다. 정말 대단하네요. 꿈의 직장이라는 말이...

**민준**
(말을 자르며)
꿈보다는 현실을 봐야죠. 여긴 전쟁터니까. 우리가 만들 기술이 얼마나 큰돈이 될지, 강지훈 씨는 상상도 못 할 겁니다.

민준의 차가운 말에 지훈의 표정이 미묘하게 굳는다. 그때, 하이힐 소리와 함께 이서연(33)이 다가온다. 자신감 넘치는 걸음걸이, 지적인 분위기.

**서연**
두 분, 여기서 다 만나셨네요. 강지훈 개발자님, 맞으시죠? 오시느라 고생 많으셨어요. 전 프로젝트 총괄을 맡은 이서연 PD입니다.

**지훈**
아, 네. 반갑습니다.

**서연**
(민준을 보며)
최 팀장님, 개발자님 오시자마자 너무 군기 잡는 거 아니에요?

**민준**
(피식 웃으며)
워밍업입니다, 이 PD님. 프로젝트의 무게감을 알려줘야죠. 감상에 젖어있을 시간이 없다는 걸.

**서연**
(지훈에게 부드럽게)
저희 팀장님 화법이 좀 직설적이라. (다시 민준에게) 그래도 아이디어는 감성에서 나오는 법이죠. 자, 두 분. 그만 서서 얘기하고 회의실로 가시죠. 박 이사님께서 기다리십니다. 앞으로 펼쳐질 미래에 대해서, 뜨겁게 한번 싸워보자고요.

서연이 앞장서서 걷는다.
지훈은 기대감 어린 눈으로, 민준은 냉소적인 눈으로, 서로 다른 생각을 품은 채 그녀의 뒤를 따른다. 세 사람의 발걸음이 향하는 곳 저편으로 'MBC Golden Mouth' 홀의 입구가 보인다. 이제 막, 거대한 서사가 시작되려 한다.

In [ ]:
#    config=GenerateContentConfig(
#        system_instruction=system_instruction,
#        #thinking_config=thinking_config,
#    ),

In [21]:
prompt="""Vertex AI Gemini API를 통해 Gemini 2.5 Pro가 생성한 대본 초안이 있어.
          생성된 대본 초안은 "files/mbc-draft_soap_drama_planning_idea.txt" 파일에 저장됨.
          Gemini 2.5 Pro의 API를 호출해서 이 텍스트 파일의 내용을 오디오 파일로 변환하고 싶어.
          변환된 오디오 파일은 "files/mbc-draft_soap_drama_planning_idea.mp3"에 저장해줘.
          파이썬 코드를 생성해줘. 
          주피터 랩에서 실행할 예정이고 GCP_PROJECT_ID 등은 이미 앞에서 설정되었음.
          만약 가능하면 google-cloud-aiplatform 라이브러리 대신 google-genai 라이브러리를 쓰고 싶어.
"""
response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
)
display(Markdown(response.text))

네, 알겠습니다. 요청하신 대로 `google-genai` 라이브러리를 사용하여 Gemini API의 Text-to-Speech(TTS) 기능을 호출하는 Python 코드를 생성해 드리겠습니다.

주피터 랩 환경에서 바로 실행할 수 있도록 구성했으며, 코드에 대한 상세한 설명도 함께 포함했습니다.

---

### 사전 준비사항

코드를 실행하기 전에 다음 사항을 확인해주세요.

1.  **라이브러리 설치:** `google-genai` 라이브러리가 설치되어 있어야 합니다.
    ```bash
    pip install google-generativeai
    ```

2.  **인증 설정:** GCP 프로젝트 ID 설정 외에, API를 호출할 수 있도록 인증이 완료되어야 합니다. 주피터 랩과 같은 로컬 환경에서는 다음 명령어를 터미널에서 실행하는 것이 가장 일반적입니다.
    ```bash
    gcloud auth application-default login
    ```
    이 명령어를 실행하면 브라우저가 열리고 Google 계정으로 로그인하여 인증을 완료할 수 있습니다.

3.  **파일 준비:** 주피터 랩 노트북 파일이 있는 위치를 기준으로 `files`라는 폴더를 만들고, 그 안에 `mbc-draft_soap_drama_planning_idea.txt` 파일을 저장해주세요.

    ```
    your-notebook-directory/
    ├── your_notebook.ipynb
    └── files/
        └── mbc-draft_soap_drama_planning_idea.txt
    ```

---

### Python 코드 (Text-to-Speech 변환)

아래 코드를 주피터 랩의 셀에 복사하여 실행하세요.

```python
import google.generativeai as genai
import os

# --- 설정 ---
# 입력 텍스트 파일과 출력 오디오 파일의 경로를 지정합니다.
INPUT_TEXT_FILE = "files/mbc-draft_soap_drama_planning_idea.txt"
OUTPUT_AUDIO_FILE = "files/mbc-draft_soap_drama_planning_idea.mp3"

# 사용할 음성을 선택합니다. (한국어 음성 예시)
# 'ko-KR-Standard-A': 여성
# 'ko-KR-Standard-B': 남성
# 'ko-KR-Standard-C': 여성
# 'ko-KR-Standard-D': 남성
SELECTED_VOICE = "ko-KR-Standard-A" 

# --- 코드 실행 ---

print("TTS 변환을 시작합니다.")

# 1. 출력 디렉터리가 없으면 생성합니다.
output_dir = os.path.dirname(OUTPUT_AUDIO_FILE)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"'{output_dir}' 디렉터리를 생성했습니다.")

try:
    # 2. 대본 텍스트 파일을 읽습니다. (한국어 처리를 위해 encoding='utf-8' 지정)
    with open(INPUT_TEXT_FILE, "r", encoding="utf-8") as f:
        text_content = f.read()
    print(f"✅ '{INPUT_TEXT_FILE}' 파일 읽기 완료.")
    # print(f"\n--- 읽어온 텍스트 내용 ---\n{text_content[:200]}...\n--------------------------\n")

    # 3. Gemini Text-to-Speech API를 호출합니다.
    print(f"Gemini API 호출 중... (음성: {SELECTED_VOICE})")
    response = genai.text_to_speech(
        model="models/text-to-speech",  # TTS 전용 모델을 사용합니다.
        text=text_content,
        voice=SELECTED_VOICE,
    )
    print("✅ API로부터 오디오 데이터 수신 완료.")

    # 4. API로부터 받은 오디오 데이터를 MP3 파일로 저장합니다.
    # 'wb'는 'write binary' 모드를 의미하며, 오디오/이미지 등 바이너리 파일 저장 시 사용합니다.
    with open(OUTPUT_AUDIO_FILE, "wb") as f:
        f.write(response.audio_content)
    
    print(f"🎉 성공! 오디오 파일이 '{OUTPUT_AUDIO_FILE}' 경로에 저장되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: 입력 파일 '{INPUT_TEXT_FILE}'을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
except Exception as e:
    print(f"❌ TTS 변환 중 오류가 발생했습니다: {e}")

```

### 코드 상세 설명

1.  **라이브러리 임포트**:
    *   `google.generativeai as genai`: Gemini API와 상호작용하기 위한 메인 라이브러리입니다.
    *   `os`: 파일 경로를 다루고, 출력 폴더(`files/`)가 없을 경우 자동으로 생성하기 위해 사용합니다.

2.  **설정 (Configuration)**:
    *   `INPUT_TEXT_FILE`: 읽어올 대본 텍스트 파일의 경로입니다.
    *   `OUTPUT_AUDIO_FILE`: 생성될 MP3 오디오 파일의 저장 경로입니다.
    *   `SELECTED_VOICE`: 오디오를 생성할 음성을 선택합니다. 주석에 있는 한국어 음성 코드(`ko-KR-Standard-A` 등) 중에서 원하는 것을 선택하여 변경할 수 있습니다.
        *   더 많은 음성 옵션은 [공식 문서](https://ai.google.dev/gemini-api/docs/models/tts#available-voices)에서 확인할 수 있습니다.

3.  **출력 디렉터리 확인**: 코드를 실행했을 때 `files` 폴더가 없다면 오류가 발생합니다. `os.path.exists`와 `os.makedirs`를 사용해 폴더가 없으면 자동으로 생성하여 편의성을 높였습니다.

4.  **텍스트 파일 읽기**:
    *   `with open(...)` 구문을 사용하여 파일을 안전하게 열고, 작업이 끝나면 자동으로 닫습니다.
    *   `encoding="utf-8"`은 한글이 깨지지 않고 올바르게 읽히도록 하기 위한 필수적인 옵션입니다.

5.  **Gemini TTS API 호출**:
    *   `genai.text_to_speech()` 함수를 호출하여 텍스트를 음성으로 변환합니다.
    *   `model`: `models/text-to-speech`라는 TTS 전용 모델명을 지정합니다.
    *   `text`: 파일에서 읽어온 대본 내용(`text_content`)을 전달합니다.
    *   `voice`: 위에서 설정한 `SELECTED_VOICE`를 전달합니다.

6.  **오디오 파일 저장**:
    *   API 응답(`response`) 객체에는 `audio_content`라는 속성에 오디오 데이터가 바이너리(binary) 형태로 담겨 있습니다.
    *   `open` 함수를 `'wb'` (write binary) 모드로 호출하여 이 바이너리 데이터를 파일에 그대로 씁니다. 이렇게 하면 MP3 파일이 생성됩니다.

7.  **예외 처리**: `try...except` 블록을 사용하여 파일이 존재하지 않거나 API 호출 중 문제가 발생했을 때 프로그램이 중단되지 않고 적절한 오류 메시지를 출력하도록 했습니다.

In [22]:
# --- 설정 ---
# 입력 텍스트 파일과 출력 오디오 파일의 경로를 지정합니다.
INPUT_TEXT_FILE = "files/mbc-draft_soap_drama_planning_idea.txt"
OUTPUT_AUDIO_FILE = "files/mbc-draft_soap_drama_planning_idea.mp3"

# 사용할 음성을 선택합니다. (한국어 음성 예시)
# 'ko-KR-Standard-A': 여성
# 'ko-KR-Standard-B': 남성
# 'ko-KR-Standard-C': 여성
# 'ko-KR-Standard-D': 남성
SELECTED_VOICE = "ko-KR-Standard-A" 

# --- 코드 실행 ---

print("TTS 변환을 시작합니다.")

# 1. 출력 디렉터리가 없으면 생성합니다.
output_dir = os.path.dirname(OUTPUT_AUDIO_FILE)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"'{output_dir}' 디렉터리를 생성했습니다.")

try:
    # 2. 대본 텍스트 파일을 읽습니다. (한국어 처리를 위해 encoding='utf-8' 지정)
    with open(INPUT_TEXT_FILE, "r", encoding="utf-8") as f:
        text_content = f.read()
    print(f"✅ '{INPUT_TEXT_FILE}' 파일 읽기 완료.")
    # print(f"\n--- 읽어온 텍스트 내용 ---\n{text_content[:200]}...\n--------------------------\n")

    # 3. Gemini Text-to-Speech API를 호출합니다.
    print(f"Gemini API 호출 중... (음성: {SELECTED_VOICE})")
    response = genai.text_to_speech(
        model="models/text-to-speech",  # TTS 전용 모델을 사용합니다.
        text=text_content,
        voice=SELECTED_VOICE,
    )
    print("✅ API로부터 오디오 데이터 수신 완료.")

    # 4. API로부터 받은 오디오 데이터를 MP3 파일로 저장합니다.
    # 'wb'는 'write binary' 모드를 의미하며, 오디오/이미지 등 바이너리 파일 저장 시 사용합니다.
    with open(OUTPUT_AUDIO_FILE, "wb") as f:
        f.write(response.audio_content)
    
    print(f"🎉 성공! 오디오 파일이 '{OUTPUT_AUDIO_FILE}' 경로에 저장되었습니다.")

except FileNotFoundError:
    print(f"❌ 오류: 입력 파일 '{INPUT_TEXT_FILE}'을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
except Exception as e:
    print(f"❌ TTS 변환 중 오류가 발생했습니다: {e}")

TTS 변환을 시작합니다.
✅ 'files/mbc-draft_soap_drama_planning_idea.txt' 파일 읽기 완료.
Gemini API 호출 중... (음성: ko-KR-Standard-A)
❌ TTS 변환 중 오류가 발생했습니다: module 'google.genai' has no attribute 'text_to_speech'


### 일반 URL에서 오디오 보내기
이 예는 [Kubernetes Podcast]](https://kubernetespodcast.com/) 에피소드의 오디오입니다.

오디오 파일을 실행하려면 다음 링크를 클릭하세요.
- [https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3](https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3)
- 크롬 브라우저의 새 탭이 열리고, 재생할 수 있습니다.

In [9]:
#prompt="Write a summary of this podcast episode."
prompt="이 팟캐스트 에피소드를 요약한 다음, 한국어로 번역해죠."
audio_file_uri="https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3"
mime_type="audio/mpeg"

In [8]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri=audio_file_uri,
            mime_type=mime_type,
        ),
        prompt,
    ],
    config=GenerateContentConfig(
        audio_timestamp=True,
        #thinking_config=thinking_config,
    ),
)

display(Markdown(response.text))

이 팟캐스트 에피소드는 KubeCon + CloudNativeCon North America 2024를 요약하며, 행사 주요 발표를 시작으로 참석자들과의 인터뷰를 통해 그들의 경험과 관찰된 트렌드를 다룹니다.

**주요 발표 및 뉴스:**
*   Cert-Manager와 Dapr가 CNCF 졸업 프로젝트가 되었습니다.
*   Istio 1.20이 출시되었으며, Ambient Mesh는 일반 가용성(GA)을 달성했습니다.
*   CNCF는 특허 괴물(patent trolls)에 맞서 싸우기 위한 Cloud Native Heroes Challenge를 시작했습니다.
*   2025년 행사 라인업에는 5개의 KubeCon + CloudNativeCon 행사, 1개의 오픈 소스 SecurityCon, 그리고 30개의 Kubernetes Community Days가 포함됩니다.
*   세 가지 새로운 클라우드 네이티브 자격증(Certified Backstage Associate, OpenTelemetry Certified Associate, Kyverno Certified Associate)이 발표되었습니다.
*   리눅스 재단은 내년부터 주요 Kubernetes 자격증(CKA, CKS, CKAD) 및 LFS 시험의 응시료를 10% 인상한다고 발표했습니다.
*   WasmCloud는 CNCF의 인큐베이팅 프로젝트로 합류했습니다.
*   Spectro Cloud는 Kubernetes 관리 솔루션을 위해 시리즈 C 펀딩으로 7천5백만 달러를 유치했습니다.
*   Solo.io는 자사의 Gloo API Gateway를 CNCF에 기증할 예정입니다.

**참석자들의 통찰력 및 트렌드:**
참석자들은 이번 행사의 주요 목표가 동료 기여자들과의 연결(특히 대면 만남), 클라우드 네이티브 기술과의 AI/ML 통합 학습, 진화하는 보안 트렌드 이해, 그리고 성능 최적화 탐색이었다고 강조했습니다.

관찰된 주요 트렌드는 다음과 같습니다:
*   **AI/ML 통합:** AI 워크로드를 클라우드 네이티브 플랫폼, 특히 Kubernetes에 통합하는 데 상당한 초점이 맞춰졌습니다.
*   **보안:** 워크로드 강화, 종단 간(end-to-end) 보안, 취약점 관리 및 새로운 보안 도구에 대한 논의가 주된 주제였습니다. 이는 기조연설(2일차)에서도 반영된 테마였습니다.
*   **커뮤니티 및 협업:** 기여자 서밋 및 SIG 모임에서의 대면 상호작용, 연결 증진, 지식 공유의 가치가 강조되었습니다.
*   **성능 최적화:** 애플리케이션 성능을 측정하고 분석하는 도구에 대한 관심이 높았습니다.
*   **특정 기술:** Istio Ambient Mesh의 GA 달성과 Envoy 기반 애플리케이션 게이트웨이 사용 증가가 특히 언급되었습니다.